In [1]:
import os.path as osp
import json
import numpy as np

In [2]:
splits = ['miniv', 'val', 'test', 'train']

ref_tasks = []
for s in splits:
    print('loading refer_filtered_instance_%s.json' % s)
    with open('/mnt/nfs/work1/elm/chenyun/PhraseCutPano/data/refvg/amt_result/refer_filtered_instance_%s.json' % s, 'r') as f:
        ref_tasks += json.load(f)


loading refer_filtered_instance_miniv.json
loading refer_filtered_instance_val.json


loading refer_filtered_instance_test.json


loading refer_filtered_instance_train.json


In [3]:
print(ref_tasks[0])

{'phrase_structure': {'attributes': ['seated'], 'type': 'name', 'name': 'hill', 'relations': []}, 'task_id': '2328797__3342390', 'iou': 0.10582814686158151, 'instance_boxes': [[50.505, 259.185, 150.295, 181.375]], 'iop': 1.0, 'image_id': 2328797, 'ann_ids': [3342390], 'Polygons': [[[[67.71000000000001, 305.805], [58.83, 283.605], [57.72, 275.28], [53.834999999999994, 259.185], [50.505, 264.18], [53.28, 283.05], [52.724999999999994, 288.045], [56.055, 314.685], [76.59, 387.945], [78.81, 439.005], [89.91000000000001, 439.56], [192.03, 439.56], [193.14, 346.32], [199.245, 342.435], [199.79999999999998, 338.54999999999995], [79.92, 339.66], [62.714999999999996, 285.825]]]], 'phrase': 'seated hill', 'turk_id': 'A2P7Z2TKGACY2E', 'iob': 0.10582814686158151}


In [4]:
fast_text_path = '/mnt/nfs/work1/elm/chenyun/PhraseCutPano/data/fast_text/'
ft_vocab = np.load(osp.join(fast_text_path, 'vocabulary_ft.npy'))
ft_vocab = list(ft_vocab)
ft_embeddings = np.load(osp.join(fast_text_path, 'embeddings_ft.npy'))

In [5]:
word_count = dict()
special = []


def replace_special(name):
    name = name.lower()
    i = 0
    while i < len(name):
        c = name[i]
        if (c > 'z' or c < 'a') and (c < '0' or c > '9') and c != ' ':
            if c not in special: 
                special.append(c)
            name = name[:i] + ' ' + c + ' ' + name[i+1:]
            i += 2
        i += 1
    return name


for task in ref_tasks:
    phrase = task['phrase']
    phrase = replace_special(phrase)
    words = phrase.split()
    for w in words:
        word_count[w] = word_count.get(w, 0) + 1

In [ ]:
print(len(word_count))
print(special)
ws = [w for w, c in word_count.items() if c > 20]
print(len(ws))

7810
['-', "'", ',', ':', '<', '.', '/', '?', '*', '"', '\\', '&', '\x00', '`', '!', ']', '[', '+', '@', '(', ')']
1786


In [56]:
miss = 0
for w, c in word_count.items():
    # if not re.match('^[\w-]+$', w):
    #     print(w,c)
    if w not in ft_vocab and c > 5:
        miss += 1
        print(w, c, miss)

woode 7 1
whtie 18 2
leaveless 8 3
mettalic 6 4


In [59]:
vocab = ['<PAD>', '<UNK>', '<BOS>', '<EOS>'] + list(word_count.keys())
freq =  [100] * 4 + list(word_count.items())

lookup = dict()
lookup['included'] = ['refvg_train', 'refvg_val', 'refvg_test', 'refvg_miniv']
lookup['ix_to_word'] = vocab
lookup['freq'] = freq

embeddings = np.empty((len(vocab), 300))
for i, w in enumerate(vocab):
    if w in ft_vocab:
        ft_i = ft_vocab.index(w)
        embeddings[i] = ft_embeddings[ft_i]
    else:
        embeddings[i] = np.random.randn(300) / 300.0
        print(i, w, freq[i])
lookup['embeddings'] = embeddings

0 <PAD> 100
1 <UNK> 100
2 <BOS> 100
3 <EOS> 100
777 cluter ('cluter', 1)
818 hascaptain ('hascaptain', 1)
930 parseley ('parseley', 1)
1146 rup100 ('rup100', 1)
1528 longsleeve ('longsleeve', 1)
1542 whiet ('whiet', 3)
1647 snowskating ('snowskating', 2)
1681 brandweer ('brandweer', 1)
1733 zaged ('zaged', 1)
1753 clydesdale ('clydesdale', 3)
1762 dirtless ('dirtless', 1)
1763 cololred ('cololred', 1)
1803 bouqet ('bouqet', 1)
1839 economax ('economax', 1)
1861 pink4 ('pink4', 1)
1876 machaan ('machaan', 1)
1904 yelllow ('yelllow', 3)
1937 safford ('safford', 1)
2022 watera ('watera', 1)
2023 circuluar ('circuluar', 1)
2031 skyscrapper ('skyscrapper', 1)
2086 cemet ('cemet', 2)
2101 curtaiin ('curtaiin', 2)
2103 whicker ('whicker', 2)
2129 buildiong ('buildiong', 1)
2142 reftected ('reftected', 1)
2202 eldeerly ('eldeerly', 1)
2206 sivler ('sivler', 2)
2256 popcorned ('popcorned', 1)
2288 papersmagazines ('papersmagazines', 1)
2292 parled ('parled', 1)
2316 greywhite ('greywhite', 1)
2

5379 camoflauged ('camoflauged', 1)
5395 camperdown ('camperdown', 1)
5396 cruikshank ('cruikshank', 1)
5403 cushman ('cushman', 1)
5412 stopwar ('stopwar', 1)
5413 dogpooping ('dogpooping', 1)
5415 scurffy ('scurffy', 1)
5431 unchopped ('unchopped', 1)
5444 umpre ('umpre', 1)
5447 emaculate ('emaculate', 1)
5449 whitecapping ('whitecapping', 1)
5460 bedig ('bedig', 1)
5461 hobie ('hobie', 1)
5462 turquois ('turquois', 1)
5470 jacke ('jacke', 1)
5494 sitance ('sitance', 1)
5499 brunett ('brunett', 1)
5510 ungarnished ('ungarnished', 1)
5514 teis ('teis', 1)
5516 white3 ('white3', 1)
5526 cocrete ('cocrete', 1)
5532 cheeseless ('cheeseless', 1)
5534 rippl ('rippl', 1)
5536 archd ('archd', 1)
5539 flippable ('flippable', 1)
5543 bgrilled ('bgrilled', 1)
5545 borwn ('borwn', 1)
5549 woodent ('woodent', 1)
5558 prcelean ('prcelean', 1)
5559 merceded ('merceded', 1)
5571 refigerator ('refigerator', 1)
5574 splasing ('splasing', 1)
5601 stanging ('stanging', 1)
5609 banboo ('banboo', 1)
5626

7610 colourlless ('colourlless', 1)
7617 burdungy ('burdungy', 1)
7621 cumulous ('cumulous', 1)
7622 wooster ('wooster', 1)
7632 terrcotta ('terrcotta', 1)
7635 picckup ('picckup', 1)
7638 sadwich ('sadwich', 1)
7640 oorange ('oorange', 1)
7643 writig ('writig', 1)
7660 polypropelene ('polypropelene', 2)
7663 lavander ('lavander', 1)
7671 shutterless ('shutterless', 1)
7687 multiprint ('multiprint', 1)
7700 airaustral ('airaustral', 1)
7706 windowsil ('windowsil', 1)
7714 boundraries ('boundraries', 1)
7715 blowdrying ('blowdrying', 1)
7740 grinzing ('grinzing', 1)
7742 shortsleeve ('shortsleeve', 1)
7745 resing ('resing', 1)
7748 tranclucent ('tranclucent', 1)
7750 goant ('goant', 1)
7755 polkadots ('polkadots', 1)
7757 adirondack ('adirondack', 1)
7762 palte ('palte', 1)
7776 bightly ('bightly', 1)
7782 trainless ('trainless', 1)
7797 gron ('gron', 1)
7798 grees ('grees', 1)
7799 purprle ('purprle', 1)
7800 outstreched ('outstreched', 1)
7802 plaer ('plaer', 1)
7809 lincon ('lincon',

In [60]:
np.save('/mnt/nfs/work1/elm/chenyun/PhraseCutPano/data/fast_text/lookup_refvg_all.npy', lookup)